# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [41]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from io import StringIO
from sql_queries import *

In [42]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [43]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def insert_from_dataframe(df, table):
    sio = StringIO()
    sio.write(df.to_csv(index=None, header=None, sep="\t"))  # Write the Pandas DataFrame as a csv to the buffer
    sio.seek(0)  # Be sure to reset the position to the start of the stream

    cur.copy_from(sio, table, columns=df.columns, sep="\t", null="")
    conn.commit()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [19]:
song_files = get_files('data/song_data/')
type(song_files)

list

In [20]:
song_files_iterator = iter(song_files)
file_path = next(song_files_iterator)
df = pd.read_json(file_path, lines=True)
type(df)

pandas.core.frame.DataFrame

In [21]:
for song_file in song_files:
    df = df.append(pd.read_json(song_file, lines=True), ignore_index=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
1,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
2,ARB29H41187B98F0EF,41.88415,Chicago,-87.63241,Terry Callier,342.56934,1,SOGNCJP12A58A80271,Do You Finally Need A Friend,1972
3,ARGIWFO1187B9B55B7,NaN,,NaN,Five Bolt Main,225.09669,1,SOPSWQW12A6D4F8781,Made Like This (Live),0
4,ARH4Z031187B9A71F2,40.73197,"Newark, NJ",-74.17418,Faye Adams,156.39465,1,SOVYKGO12AB0187199,Crazy Mixed Up World,1961


## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [44]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
artist_data = artist_data.drop_duplicates()
artist_data = artist_data.replace(np.nan, '', regex=True)
artist_data.columns = ['artist_id', 'name', 'location', 'latitude', 'longitude']
artist_data.head()

,artist_id,name,location,latitude,longitude
0,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.8262,-74.48
2,ARB29H41187B98F0EF,Terry Callier,Chicago,41.8841,-87.6324
3,ARGIWFO1187B9B55B7,Five Bolt Main,,,
4,ARH4Z031187B9A71F2,Faye Adams,"Newark, NJ",40.732,-74.1742
5,AR5KOSW1187FB35FF4,Elena,Dubai UAE,49.8039,15.4749


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [45]:
insert_from_dataframe(artist_data, 'artists')

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [47]:
song_data = df[['song_id','title', 'artist_id', 'year', 'duration']]
song_data = song_data.drop_duplicates()
song_data = song_data.replace(np.nan, '', regex=True)
song_data.head()

,song_id,title,artist_id,year,duration
0,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
2,SOGNCJP12A58A80271,Do You Finally Need A Friend,ARB29H41187B98F0EF,1972,342.56934
3,SOPSWQW12A6D4F8781,Made Like This (Live),ARGIWFO1187B9B55B7,0,225.09669
4,SOVYKGO12AB0187199,Crazy Mixed Up World,ARH4Z031187B9A71F2,1961,156.39465
5,SOZCTXZ12AB0182364,Setanta matins,AR5KOSW1187FB35FF4,0,269.58322


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [48]:
insert_from_dataframe(song_data, 'songs')

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files('data/log_data')

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.